블록체인에서 주문을 하고, 그 주문내역을 로컬 파일에 작성하세요.

블록체인으로 주문이 전송되면 이벤트가 발생하도록 프로그램하고, 주문내역은 로컬파일에 저장됩니다.

* 강의자료 OrderEvent.sol을 참조해서 구현하세요.

	* order(상품항목, 주문개수, 단가, 배송지) 함수를 작성. '개수 x 단가'로 주문금액을 계산하고 이 금액만큼 입금되지 않으면 예외발생.

	* getBalance() 함수를 작성. 최초 프로그램 생성자 owner만 읽을 수 있게 modfier isOwner 작성.

### (1) REMIX에서 주문하고, 이벤트 발생하는지 보이도록 콘솔 출력 붙여넣으세요.

In [1]:
%%writefile ./src/Order.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract Order {
    address payable owner;

    event OrderLog(string);
    event OrderLog(bytes2 _itemId, uint _value);
    event OrderLog(uint256 timestamp);
    event OrderLog(address indexed _from, bytes2 _itemId, uint quantity, uint unitPrice, string addr);

    constructor() public payable {
        owner = msg.sender;
    }

    function order(bytes2 _itemId, uint quantity, uint unitPrice, string memory addr) public payable {
        uint256 orderTime = block.timestamp;
        uint256 orderAmount = quantity * unitPrice;

        require(msg.value == orderAmount);
        emit OrderLog("Ordered");
        emit OrderLog(orderTime);
        emit OrderLog(msg.sender, _itemId, quantity, unitPrice, addr);
    }

    function getBalance() view public isOwner returns(uint) {
        return address(this).balance;
    }

    modifier isOwner() {
        if(msg.sender != owner) {
            revert();
        }
        _;
    }
}

Writing ./src/Order.sol


In [ ]:
[vm]from: 0x5B3...eddC4to: Order.(constructor)value: 300 weidata: 0x608...40033logs: 0hash: 0x7bf...b379d
status	true Transaction mined and execution succeed
transaction hash	0x7bfa8c922d3e2ffad2c579209c5efa132fe53a611ad8c6cdc145314bcbeb379d
from	0x5B38Da6a701c568545dCfcB03FcB875f56beddC4
to	Order.(constructor)
gas	323263 gas
transaction cost	281098 gas 
execution cost	281098 gas 
input	0x608...40033
decoded input	{}
decoded output	 - 
logs	[]
val	300 wei

### (2) 파일로 컴파일 하세요.

In [2]:
!solc.exe --optimize --combined-json abi,bin src/Order.sol > src/Order.json

### (3) ganache에서 배포하세요.

In [5]:
%%writefile src/OrderDeploy.js
var Web3 = require('web3');
var _abiBinJson = require('./Order.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract form " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x" + _bin})
        .send({from: accounts[0], gas: 259210, function(err, transactionHash) {
            if(!err) {
                console.log("hash : " + transactionHash);
            }
        }})
    console.log("---> The contract deployed to : " + deployed.options.address);
}

deploy();

Overwriting src/OrderDeploy.js


In [6]:
!node src/OrderDeploy.js

- contract name :  [ 'src/Order.sol:Order' ]
Deploying the contract form 0xC6D6c96820dC90740fB25Ad4959b15ecb1C3d1Af
---> The contract deployed to : 0x5508535F46c426c891DE7d893D69BE71CeE64833


### (4) 주문을 3건만 하고, 이벤트를 통해 다음과 같이 출력하세요.
항목은 차례대로 주소, 상품항목번호, 주문개수, 단가, 배송지.

전송자주소, 1111, 3, 100, 20 2-gil Hongji-dong Jongro-gu Seoul

전송자주소, 1111, 5, 100, 20 2-gil Hongji-dong Jongro-gu Seoul

전송자주소, 1111, 20, 100, 20 2-gil Hongji-dong Jongro-gu Seoul

In [91]:
%%writefile src/OrderUse.js
var Web3 = require('web3');
var fs = require('fs');
var _abiBinJson = require('./Order.json');
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function doIt() {
    var _order = new web3.eth.Contract(_abiArray, '0x5508535F46c426c891DE7d893D69BE71CeE64833');
    const accounts = await web3.eth.getAccounts();
    console.log("Account : " + accounts[0]);
    
    var event = _order.events.OrderLog(
        {fromBlock: 0}, function(error, result) {
            if(!error) {
                var log = JSON.stringify(result.returnValues);
                console.log("Event fired : " + log);
                fs.appendFile("src/OrderEvent.txt", log, "utf-8", function(e) {
                    if(!e) {
                        console.log(">> Writing to file");
                    }
                });
            }
            else {
                console.log(error);
            }
        }
    );
    
    var value;
    my = await _order.methods.order("0x1111", 3, 100, "20 2-gil Hongji-dong Jongro-gu Seoul")
            .send({from: accounts[0], gas: 100000, value: 300});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    
    my = await _order.methods.order("0x1111", 5, 100, "20 2-gil Hongji-dong Jongro-gu Seoul")
            .send({from: accounts[0], gas: 100000, value: 500});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    
    my = await _order.methods.order("0x1111", 20, 100, "20 2-gil Hongji-dong Jongro-gu Seoul")
            .send({from: accounts[0], gas: 100000, value: 2000});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    
    process.exit(1);
}

doIt();

Overwriting src/OrderUse.js


In [92]:
!node src/OrderUse.js

- contract name :  [ 'src/Order.sol:Order' ]
Account : 0xC6D6c96820dC90740fB25Ad4959b15ecb1C3d1Af
Event fired : {"0":"0xC6D6c96820dC90740fB25Ad4959b15ecb1C3d1Af","1":"0x1111","2":"3","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0xC6D6c96820dC90740fB25Ad4959b15ecb1C3d1Af","_itemId":"0x1111","quantity":"3","unitPrice":"100","addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}
Event fired : {"0":"0xC6D6c96820dC90740fB25Ad4959b15ecb1C3d1Af","1":"0x1111","2":"5","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0xC6D6c96820dC90740fB25Ad4959b15ecb1C3d1Af","_itemId":"0x1111","quantity":"5","unitPrice":"100","addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}
Event fired : {"0":"0xC6D6c96820dC90740fB25Ad4959b15ecb1C3d1Af","1":"0x1111","2":"20","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0xC6D6c96820dC90740fB25Ad4959b15ecb1C3d1Af","_itemId":"0x1111","quantity":"20","unitPrice":"100","addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}
Event fired : {"0":"0xC6D6c968

### (5) 파일에 저장된 파일내역을 출력하세요 (주피터 노트북에서 !type src/OrderEvent.txt)

In [93]:
!type src\OrderEvent.txt

{"0":"0xC6D6c96820dC90740fB25Ad4959b15ecb1C3d1Af","1":"0x1111","2":"20","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0xC6D6c96820dC90740fB25Ad4959b15ecb1C3d1Af","_itemId":"0x1111","quantity":"20","unitPrice":"100","addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}{"0":"0xC6D6c96820dC90740fB25Ad4959b15ecb1C3d1Af","1":"0x1111","2":"3","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0xC6D6c96820dC90740fB25Ad4959b15ecb1C3d1Af","_itemId":"0x1111","quantity":"3","unitPrice":"100","addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}{"0":"0xC6D6c96820dC90740fB25Ad4959b15ecb1C3d1Af","1":"0x1111","2":"5","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0xC6D6c96820dC90740fB25Ad4959b15ecb1C3d1Af","_itemId":"0x1111","quantity":"5","unitPrice":"100","addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}{"0":"0xC6D6c96820dC90740fB25Ad4959b15ecb1C3d1Af","1":"0x1111","2":"5","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0xC6D6c96820dC90740fB25Ad4959b15ecb1C3d1